This notebook shows showcases how to create a custom magnet instrument consisting of different current sources with 'cqed.utils.custom_magnet.py'. In this example we show how to use the Oxford Mercury IPS for the y and z axes, and the Yokogawa GS610 for the x axis.out of this. 

We then introduce the pysweep sweep objects we have implemented for the magnet in 'cqed.custom_pysweep_functions.magnet.py'.




# Imports

In [1]:
import pysweep
import pysweep.databackends.debug
import pysweep.databackends.list_backend
import pysweep.databackends.qcodes
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
from pysweep.databackends.base import DataParameterFixedAxis
from pysweep.databackends.base import DataParameter
import pysweep.convenience as conv

import qcodes as qc
from qcodes.dataset.measurements import Measurement
import qcodes.dataset.plotting

from cqed.utils.datahandling import create_local_dbase_in
import cqed.custom_pysweep_functions.vna as cvna
import cqed.custom_pysweep_functions.magnet as cmgnt

import numpy as np
import matplotlib.pyplot as plt

# Set up station and QCoDeS database

## Set up measurement station

Choose the appropriate file for your measurement setup. The repo contains templates with commonly used instruments for K1 and K2.

In [ ]:
station = qc.Station(config_file=r'D:\OneDrive\Setups\LK2\LK2_code\code\cqed\station_init\LK2.station.yaml')

## Set up instruments

For this example we are only going to be using the ZNB20 VNA from R&S.

In [ ]:
vna = station.load_instrument('vna')

vna.add_channel('S21')
station.vna.S21.power(-50)
station.vna.S21.start(4e9)
station.vna.S21.stop(8.5e9)
station.vna.S21.bandwidth(1e3)
station.vna.S21.npts(10001)

# Automatically remove the cable delay from the phase signal
# Should be checked in large frequency range and with higer readout power
# Often can be fine-tuned a bit better by hand.
station.vna.S21.set_electrical_delay_auto()

In [ ]:
mgnt = station.load_instrument('mgnt', field_limits=cmgnt.field_limit)
mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))
print('current X-field:', station.mgnt.x_measured(), 'T')
print('current Y-field:',station.mgnt.y_measured(), 'T')
print('current Z-field:',station.mgnt.z_measured(), 'T')

In [ ]:
yoko_x = station.load_instrument('yoko_x')
print('Status: {}'.format(yoko_x.output()))
print('Mode: {}'.format(yoko_x.source_mode()))
yoko_x.output('off')
yoko_x.source_mode('CURR')
yoko_x.current_range(3.0) #0.5

## Pass measurement station to pysweep 


In [ ]:
pysweep.STATION = station

## Name and initialize database

In [ ]:
# To place the database in D:/Data/MyExperiment/ use:
folder = 'MyExperiment'

# To place the database in a more nested folder structure, e.g. D:/Data/MyExperiment/TestSample/FirstMeasurements/, use:
# folder = 'MyExperiment/TestSample/FirstMeasurements'

# Name your database
database_name = 'yyyy_mm_dd_Run1'

create_local_dbase_in(folder_name=folder, db_name='%s.db' %(database_name))

qc.config['core']['db_location']

# Custom magnet

Lets now build a custom magnet object with the Oxford Mercury IPS for the y and z axes, and the Yokogawa GS610 for the x axis.out of this. 

This entails wrapping the Yokogawa into an instrument that has the get and set commands as desired by the magnetic field pysweep sweep objects.

In [ ]:
customyokox = cqed.general_tools.custom_magnet.CustomGS210(name='YokoGS610X', instrument=yoko_x, coil_constant=58.618, step=5e-6*59.081, delay=5e-3)

The Oxford Mercury IPS already has its get and set commands in the structure used in our sweep objects, so that does not need to be wrapped.

We then combine them into the meta instrument, add it to the station, and pass it on to the magnetic field pysweep module in 'cqed.custom_pysweep_functions.magnet.py'. 

In [ ]:
custommagnet = cqed.general_tools.custom_magnet.CustomMagnet(name='CustomMagnet', x_source=customyokox, y_source=mgnt.GRPY, z_source=mgnt.GRPZ)
station.add_component(custommagnet)

mgnt_sweeps = cmgnt.Magnet(instrument=custommagnet)
#if one does not have a custom magnet set up, then just use cmgnt.Magnet(instrument=mgnt)!

This instrument now has many of the functions we are used to from the Oxford magnet driver, although not every function has already been implemented.

In [ ]:
# Get current field strength of x, y, and z component.
print(station.custommagnet.x_measured())
print(station.custommagnet.y_measured())
print(station.custommagnet.z_measured())

In [ ]:
# Set all target field components to some (reasonable) value and ramp magnetic field.
# After ramping, check field strength
station.custommagnet.x_target(0.0e-3)
station.custommagnet.y_target(0.0e-3)
station.custommagnet.z_target(0.1e-3)

station.mgnt.ramp(mode='safe')

Let us now look at the pysweep objects we have created that use this custom magnet object. 

# Custom magnet sweeps

The sweep functions below either explicitly use x, y, and z coordinates, or they calculate x, y, and z components from the spherical component input. This is done for greater control of the actual sweeping direction of the magnet.

Implicit in the functions are also a number of failsafes and conventions. A field limit with a default value of r = 1.5 T is implemented that has to be manualy overriden in order to protect the TWPA shield. Furthermore, we use the ISO 80000-2:2009 physics convention for the (r, theta, phi) <--> (x, y, z) definition. In this convention the coordinates have to satisfy
- theta: inclination angle, unit: degrees, 0 <= theta <= 180
- phi: the azimuth (in plane) angle, unit: degrees, range: 0 <= phi <= 360
- r: radius, unit: Tesla, range: 0 <= r <= max_field_strength

In what follows, we also make use of the '+' functionality of pysweep to record the measured magnetic field components (cartesian or spherical) along with the desired setpoints.

## X/Y/Z sweep

In [ ]:
# Function called before beginning of measurement loop.
# This is a good place to define the VNA sweep parameters. Can also be made a function with changable parameters.

def init_measurement(d):
    # some example parameters that could be set in the init function
    station.vna.rf_on()
    station.vna.S21.start(4e9)
    station.vna.S21.stop(5e9)
    station.vna.S21.npts(1001)
    
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1e3)
    station.vna.S21.avg(10)
    
    print('Starting measurement.')
    
# Function called after the end of the measurement.
def end_measurement(d):
    print('Measurement finished.')

# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt_X', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_components + cmgnt.measure_magnet_components_sph + cvna.cvna.measure_linear_sweep,
                       cmgnt.sweep_x(points = np.arange(0, 10e-3, 1e-3)),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))

### Phi sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-phi', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_components + cmgnt.measure_magnet_components_sph + cvna.cvna.measure_linear_sweep, 
                  cmgnt.sweep_phi(r = 3.0e-3, theta = 90, points = np.arange(0, 360, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

### Theta sweep

In this example we override the conventions to allow sweeping to negative angles in a single continous sweep.

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-theta', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_components_sph + cvna.cvna.measure_linear_sweep, 
                  cmgnt.sweep_theta(r = 3.0e-3, phi = 90, points = np.arange(-10, 11, 1), use_conventions=False),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

### r sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-r', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_components + cvna.cvna.measure_linear_sweep, 
                  cmgnt.sweep_r(phi = 90, theta = 0, points = np.arange(0, 20e-3, 2e-3)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))